In [4]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, datasets, regularizers
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
import keras_tuner
import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots
import tensorboard
import tempfile
import pathlib

## Configurações para melhorar a Pipeline de entrada

In [ ]:
devices = tf.config.list_physical_devices()
print(devices)
a=tf.random.normal([100,100])
b=tf.random.normal([100,100])
c = a*b

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


In [ ]:
import tensorflow as tf
tf.debugging.set_log_device_placement(False)
# Verifique as GPUs disponíveis
print("GPUs disponíveis:", tf.config.list_physical_devices('GPU'))



In [ ]:
valor_perda, valor_precisao = model_lenet5.evaluate(test_generator)
print('\nPrecisão na classificação das amostras de teste:', valor_precisao)

## LeNet5

In [1]:
!pip install git+https://github.com/tensorflow/docs

  Cloning https://github.com/tensorflow/docs to c:\users\cmarc\appdata\local\temp\pip-req-build-s0eycgwq
  Resolved https://github.com/tensorflow/docs to commit 3b8eeeeddb85f606812874d2aa45823227afc6bf
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached astor-0.8.1-py2.py3-none-any.whl (27 kB)
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Obtaining dependency information for nbformat from https://files.pythonhosted.org/packages/01/e5/322b31448ba6b0ed6de740306367e85d8da2af0d91e67f7a2860bdf87f72/nbformat-5.9.1-py3-none-any.whl.metadata
  Obtaining dependency information for protobuf>=3.12 from https://files.pythonhosted.org/packages/80/70/dc63d340d27b8ff22022d7dd14b8d6d68b479a003eacdc4507150a286d9a/protobuf-4.23.4-cp310-abi3-win_amd64.whl.metadata
  Obtaining dependency information for pyyaml from https://files.pythonhosted.org/packages/b3/34/65bb4b2d790804496

  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/docs 'C:\Users\cmarc\AppData\Local\Temp\pip-req-build-s0eycgwq'


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import image_dataset_from_directory

num_batches = 10
classes = ["classe1","classe2","classe3","classe4","classe5"]
image_size = 400

train_data_dir = "to_torch/train/"
test_data_dir = "to_torch/test/"
valid_data_dir = "to_torch/valid/"


train_data = image_dataset_from_directory(train_data_dir, batch_size=num_batches, class_names=classes, image_size=(image_size, image_size))
valid_data = image_dataset_from_directory(valid_data_dir, batch_size=num_batches, class_names= classes, image_size= (image_size, image_size))
test_data = image_dataset_from_directory(test_data_dir, batch_size= num_batches, image_size= (image_size, image_size))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc1 in position 28: invalid start byte

In [ ]:
n_train = (len(train_data) * num_batches)
steps_per_epoch = n_train//num_batches

In [ ]:
# Definindo a arquitetura da LeNet-5
model_lenet5 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(6, (3, 3), activation='relu',strides= (2,2), input_shape=(image_size, image_size, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', strides=(2,2)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(120, activation='relu'),
    tf.keras.layers.Dropout(rate=0.5),
    
    tf.keras.layers.Dense(84, activation='relu'),
    tf.keras.layers.Dropout(rate=0.5),
    
    #TESTAR FUNÇAO SIGMOID (0)
    tf.keras.layers.Dense(5, activation='softmax')
])

# # Compilando o modelo
# model_lenet5.compile(optimizer='adam', 
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
#               metrics=['accuracy'])

# model_lenet5.summary()

In [ ]:
class_names = train_data.class_names
print(class_names)

In [ ]:
for image_batch, labels_batch in train_data:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

In [ ]:
import numpy as np

normalization_layer = tf.keras.layers.Rescaling(1./255)

normalized_data = train_data.map(lambda x, y,: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_data))
first_image = image_batch[0]

print(np.min(first_image), np.max(first_image))

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
class CompilerAndFit:
    def __init__(self, optmizer, path):
        self.optmizer = optmizer
        self.path = path

    def __get_lr_schedule(self):
        lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
            0.001,
            decay_steps= steps_per_epoch*100,
            decay_rate=1,
            staircase=False
            )
        return lr_schedule
    
    def __get_log_dir(self):
        if not os.path.exists(self.path):
            os.makedirs(self.path)
            print("Pasta criada com sucesso")
        else:
            print("A pasta já existe")
        return self.path

    def __get_callbacks(self):
        return [
            tfdocs.modeling.EpochDots(),
            tf.keras.callbacks.EarlyStopping(monitor='acc', patience=20, mode="max"),
            tf.keras.callbacks.TensorBoard(log_dir= self.__get_log_dir(pth)),
        ]

    def __get_optimizer(self,):
        if self.optmizer == "SGD":
            return tf.keras.optimizers.SGD(self.__get_lr_schedule)
        elif self.optmizer == "Adam":
            return tf.keras.optimizers.Adam(lr_schedule)

    def compile_and_fit(model, train_Data, validation_Data, max_epochs, callback_path,optimizer= None):
        if optimizer is None:
            optimizer = get_optimizer()
        model.compile(optimizer= optimizer,
                            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                            metrics= [
                                keras.metrics.SparseCategoricalAccuracy(name="acc"),
                                keras.metrics.SparseTopKCategoricalAccuracy(2, name="top5-acc")]
                            )
        model.summary()
        
        history = model.fit(
            train_Data,
            steps_per_epoch= steps_per_epoch,
            validation_data= validation_Data,
            epochs= max_epochs,
            callbacks= get_callbacks(callback_path)
        )
        return history


In [ ]:
history = {}
history["lenet5"] = compile_and_fit(model_lenet5,
                                           train_Data= train_data,
                                           validation_Data= valid_data,
                                           max_epochs= 100,
                                           callback_path= "logs_lenet5")

In [ ]:
plotter = tfdocs.plots.HistoryPlotter(metric = 'acc')
plotter.plot(history)

val_acc = history["lenet5"].history['val_acc']
max_val_acc = max(val_acc)

print("Maior valor de val_acc observado:", max_val_acc)

plt.axhline(max_val_acc, color='red', linestyle='dashed', label='Max val_acc')
plt.legend()

# Mostrar o gráfico
plt.show()

### LeNet5 - regularizers L2

In [ ]:
model_lenet5_l2 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(6, (3, 3), activation='relu',strides= (2,2), input_shape=(image_size, image_size, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', strides=(2,2)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(120, activation='relu',
                          kernel_regularizer= regularizers.l2(0.01)),
    tf.keras.layers.Dropout(rate=0.25),
    
    tf.keras.layers.Dense(84, activation='relu',
                          kernel_regularizer= regularizers.l2(0.01)),
    tf.keras.layers.Dropout(rate=0.25),
    
    #TESTAR FUNÇAO SIGMOID (0)
    tf.keras.layers.Dense(5, activation='softmax')
])

In [ ]:
history2 = {}
history2["lenet5_l2"] = compile_and_fit(model_lenet5_l2,
                                           train_Data= train_data,
                                           validation_Data= valid_data,
                                           max_epochs= 50,
                                           callback_path= "lenet5_l2")

In [ ]:
plotter = tfdocs.plots.HistoryPlotter(metric = 'acc')
plotter.plot(history2)

val_acc = history2["lenet5_l2"].history['val_acc']
max_val_acc = max(val_acc)

print("Maior valor de val_acc observado:", max_val_acc)

plt.axhline(max_val_acc, color='red', linestyle='dashed', label='Max val_acc')
plt.legend()

# Mostrar o gráfico
plt.show()

### LeNet5 - compiler SGD 

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    0.001,
    decay_steps= steps_per_epoch*100,
    decay_rate=1,
    staircase=False
)

def get_log_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)
        print("Pasta criada com sucesso")
    else:
        print("A pasta já existe")
    return path


def get_callbacks(path):
    return [
        tfdocs.modeling.EpochDots(),
        tf.keras.callbacks.EarlyStopping(monitor='acc', patience=20, mode="max"),
        tf.keras.callbacks.TensorBoard(log_dir= get_log_dir(path)),
    ]

def get_optimizer():
    return tf.keras.optimizers.SGD(lr_schedule)

def compile_and_fit(model, train_Data, validation_Data, max_epochs, callback_path,optimizer= None):
    if optimizer is None:
        optimizer = get_optimizer()
    model.compile(optimizer= optimizer,
                         loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                         metrics= [
                             keras.metrics.SparseCategoricalAccuracy(name="acc"),
                             keras.metrics.SparseTopKCategoricalAccuracy(2, name="top5-acc")]
                         )
    model.summary()
    
    history = model.fit(
        train_Data,
        steps_per_epoch= steps_per_epoch,
        validation_data= validation_Data,
        epochs= max_epochs,
        callbacks= get_callbacks(callback_path)
    )
    return history

model_lenet5_l2_sgd = tf.keras.Sequential([
    tf.keras.layers.Conv2D(6, (3, 3), activation='relu',strides= (2,2), input_shape=(image_size, image_size, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', strides=(2,2)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(120, activation='relu',
                          kernel_regularizer= regularizers.l2(0.01)),
    tf.keras.layers.Dropout(rate=0.5),
    
    tf.keras.layers.Dense(84, activation='relu',
                          kernel_regularizer= regularizers.l2(0.01)),
    tf.keras.layers.Dropout(rate=0.5),
    
    #TESTAR FUNÇAO SIGMOID (0)
    tf.keras.layers.Dense(5, activation='softmax')
])

In [ ]:
history3 = {}
history3["lenet5_sgd"] = compile_and_fit(model_lenet5_l2_sgd,
                                           train_Data= train_data,
                                           validation_Data= valid_data,
                                           max_epochs= 100,
                                           callback_path= "logs_lenet5")

In [ ]:
plotter = tfdocs.plots.HistoryPlotter(metric = 'acc')
plotter.plot(history3)

val_acc = history3["lenet5_sgd"].history['val_acc']
max_val_acc = max(val_acc)

print("Maior valor de val_acc observado:", max_val_acc)

plt.axhline(max_val_acc, color='red', linestyle='dashed', label='Max val_acc')
plt.legend()

# Mostrar o gráfico
plt.show()

In [ ]:
predict = model.predict_generator(test_generator)

get_best_hp(tuner_bayesian_optimization)

In [ ]:
import matplotlib.pyplot as plt

output_image = model_lenet5_l2.predict(test_data)
plt.imshow(output_image)
plt.show

### LeNet5 - Dropouts 0.25

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    0.001,
    decay_steps= steps_per_epoch*100,
    decay_rate=1,
    staircase=False
)

def get_log_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)
        print("Pasta criada com sucesso")
    else:
        print("A pasta já existe")
    return path


def get_callbacks(path):
    return [
        tfdocs.modeling.EpochDots(),
        tf.keras.callbacks.EarlyStopping(monitor='acc', patience=20, mode="max"),
        tf.keras.callbacks.TensorBoard(log_dir= get_log_dir(path)),
    ]

def get_optimizer():
    return tf.keras.optimizers.Adam(lr_schedule)

def compile_and_fit(model, train_Data, validation_Data, max_epochs, callback_path,optimizer= None):
    if optimizer is None:
        optimizer = get_optimizer()
    model.compile(optimizer= optimizer,
                         loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                         metrics= [
                             keras.metrics.SparseCategoricalAccuracy(name="acc"),
                             keras.metrics.SparseTopKCategoricalAccuracy(2, name="top5-acc")]
                         )
    model.summary()
    
    history = model.fit(
        train_Data,
        steps_per_epoch= steps_per_epoch,
        validation_data= validation_Data,
        epochs= max_epochs,
        callbacks= get_callbacks(callback_path)
    )
    return history

In [ ]:

# Definindo a arquitetura da LeNet-5
model_lenet5_drop025 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(6, (3, 3), activation='relu',strides= (2,2), input_shape=(image_size, image_size, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', strides=(2,2)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(120, activation='relu'),
    tf.keras.layers.Dropout(rate=0.25),
    
    tf.keras.layers.Dense(84, activation='relu'),
    tf.keras.layers.Dropout(rate=0.25),
    
    #TESTAR FUNÇAO SIGMOID (0)
    tf.keras.layers.Dense(5, activation='softmax')
])

In [ ]:
history4 = {}
history4["lenet5_drop025"] = compile_and_fit(model_lenet5_drop025,
                                           train_Data= train_data,
                                           validation_Data= valid_data,
                                           max_epochs= 100,
                                           callback_path= "logs_lenet5")

In [ ]:
plotter = tfdocs.plots.HistoryPlotter(metric = 'acc')
plotter.plot(history4)

val_acc = history4["lenet5_drop025"].history['val_acc']
max_val_acc = max(val_acc)

print("Maior valor de val_acc observado:", max_val_acc)

plt.axhline(max_val_acc, color='red', linestyle='dashed', label='Max val_acc')
plt.legend()

# Mostrar o gráfico
plt.show()

### LeNet5 - Dropout 0.5 + L2

In [ ]:
model_lenet5_dropout_l2 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(6, (3, 3), activation='relu',strides= (2,2), input_shape=(image_size, image_size, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', strides=(2,2)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(120, activation='relu',
                          kernel_regularizer= regularizers.l2(0.01)),
    tf.keras.layers.Dropout(rate=0.5),
    
    tf.keras.layers.Dense(84, activation='relu',
                          kernel_regularizer= regularizers.l2(0.01)),
    tf.keras.layers.Dropout(rate=0.5),
    
    #TESTAR FUNÇAO SIGMOID (0)
    tf.keras.layers.Dense(5, activation='softmax')])

In [ ]:
history5 = {}
history5["lenet5_drop_l2"] = compile_and_fit(model_lenet5_dropout_l2,
                                           train_Data= train_data,
                                           validation_Data= valid_data,
                                           max_epochs= 100,
                                           callback_path= "logs_lenet5_dropout_l2")

In [ ]:
plotter = tfdocs.plots.HistoryPlotter(metric = 'acc')
plotter.plot(history5)

val_acc = history5["lenet5_drop_l2"].history['val_acc']
max_val_acc = max(val_acc)

print("Maior valor de val_acc observado:", max_val_acc)

plt.axhline(max_val_acc, color='red', linestyle='dashed', label='Max val_acc')
plt.legend()

# Mostrar o gráfico
plt.show()

### LeNet5 - (-MaxPooling)

In [ ]:
model_lenet5_maxpooling = tf.keras.Sequential([
    tf.keras.layers.Conv2D(6, (3, 3), activation='relu',strides= (2,2), input_shape=(image_size, image_size, 3)),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', strides=(2,2)),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(120, activation='relu'),
    tf.keras.layers.Dropout(rate=0.5),
    
    tf.keras.layers.Dense(84, activation='relu'),
    tf.keras.layers.Dropout(rate=0.5),
    
    #TESTAR FUNÇAO SIGMOID (0)
    tf.keras.layers.Dense(5, activation='softmax')])

In [ ]:
history6 = {}
history6["lenet5_drop_maxp"] = compile_and_fit(model_lenet5_maxpooling,
                                           train_Data= train_data,
                                           validation_Data= valid_data,
                                           max_epochs= 100,
                                           callback_path= "logs_lenet5_dropout_l2")

In [ ]:
plotter = tfdocs.plots.HistoryPlotter(metric = 'acc')
plotter.plot(history6)

val_acc = history6["lenet5_drop_maxp"].history['val_acc']
max_val_acc = max(val_acc)

print("Maior valor de val_acc observado:", max_val_acc)

plt.axhline(max_val_acc, color='red', linestyle='dashed', label='Max val_acc')
plt.legend()

# Mostrar o gráfico
plt.show()

## TensorBoard

In [ ]:
import tensorboard
%tensorboard --logdir ./logs
